In [1]:
import statsmodels.api as sm


def smooth(y):
    #return savitzky_golay(y, window_size=2001, order=3)
    return lowess(y)

# 0.2가 제일 잘 없앴음
def lowess(y, f=0.2):
    x = np.arange(0, len(y))
    return sm.nonparametric.lowess(y, x, frac=f, it=0)[:, 1].T


import numpy as np
import pandas as pd
import os, pickle, sys
from scipy import signal


### input 설정
SRATE = 250 # 250Hz
LEN_INPUT = 20 # input 10s
LEN_PER_NRS = 60 # vital length for each NRS
OVERLAP = 2
n_aug = int((LEN_PER_NRS-LEN_INPUT)/OVERLAP) + 1 # data augmentation 개수

non_preop_list = []


input_path = '../DL_model/dataset/preprocess4/input3/'
if not os.path.exists(input_path[:-1]):
    os.mkdir(input_path[:-1])


if True:
    false_row_list_preop = []
    
    preop_ecg_path = "../../cranberry2/Preprocessing/vital_data/ECG_250Hz_preop_2min/"
    vital_path = '../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_preop_2min/'
    df_preprocess_preop = pickle.load(open('../DL_model/cache/preprocess3/input3/df_preprocess_preop_agender','rb'))
    df_demograph = pd.read_csv('../DL_model/caseids_age_gender.csv')
    

    ### test set에 해당하는 file_path
    caseid_test = pickle.load(open('../DL_model/caseid_test_new', 'rb'))
    caseid_train = pickle.load(open('../DL_model/caseid_train_new', 'rb'))
    caseid_val = pickle.load(open('../DL_model/caseid_val_new', 'rb'))
    
    non_list = []
    x_train_preop, y_train_preop = [], []
    x_test_preop, y_test_preop = [], []
    x_val_preop,y_val_preop = [], []
    age_train_preop, gender_train_preop = [], []
    age_test_preop, gender_test_preop = [], []
    age_val_preop, gender_val_preop = [], []    

    cnt = 0
    for _, row in df_preprocess_preop.iterrows():
        cnt += 1
        if cnt<=1900:
            continue
        print('loading data {}/{} ...'.format(cnt, len(df_preprocess_preop)), end='')
        row_demo = df_demograph[df_demograph['caseids']==row['caseids']].iloc[0]
        
        
        # vital data - PPG (resampling 100 Hz to 250 Hz)
        df_vital = pickle.load(open(vital_path+row['file_path'], 'rb')).reset_index()
        pleth_samp = df_vital[['Pleth']].fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        pleth_resamp = signal.resample(pleth_samp, 120*SRATE)
        ppg_per_NRS = np.full(30000, np.nan)
        ppg_per_NRS[0:len(pleth_resamp)] = pleth_resamp
        
        # vital data - ECG (250Hz)       
        try:
            ecg_samp = pickle.load(open(preop_ecg_path+row['file_path'].split(',')[2], 'rb'))
            
        except:
            non_preop_list.append(row['file_path'])
            print('no preop data')
            continue
        else:
            ecg_samp = pd.DataFrame(ecg_samp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()[0:30000]       
            ecg_per_NRS = np.full(30000,np.nan)
            ecg_per_NRS[0:len(ecg_samp)] = ecg_samp
            
            # 한 NRS에 대해 23개의 input 확인
            save_path = '../../cranberry2/Preprocessing/cache/lowess_filtered/preop_'+row['caseids']
            # 한 NRS에 대해 23개의 input 확인
            for i in range(0,30):
                # input이 전처리 통과한 경우
                if row[str(i+1)]:
                    start_idx = i*OVERLAP*SRATE # 500i
                    end_idx = (i*OVERLAP+LEN_INPUT)*SRATE # 500i + 1000

                    # input의 normalization
                    ppg_inp = np.copy(ppg_per_NRS[start_idx:end_idx])
                    if np.sum(np.isnan(ppg_inp))!=0:
                        ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()

                    pleth_inp = ppg_inp - smooth(ppg_inp)
                    pleth_inp = pleth_inp - np.nanmean(pleth_inp)

                    ecg_inp = np.copy(ecg_per_NRS[start_idx:end_idx])
                    if np.sum(np.isnan(ecg_inp))!=0:
                        ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()

                    ecg_inp = ecg_inp - smooth(ecg_inp)
                    ecg_inp = (ecg_inp - np.nanmean(ecg_inp)) / np.nanstd(ecg_inp)

                    
                    pickle.dump([pleth_inp, ecg_inp],  open(save_path+'_{}'.format(i), 'wb'))
                    # 해당 caseid가 test set에 속하는 경우
                    if row['caseids'] in caseid_test:
                        age_test_preop.append(int(row_demo['age']))
                        if row_demo['gender']=='F':
                            gender_test_preop.append(1)
                        else:
                            gender_test_preop.append(0)
                        x_test_preop.append([pleth_inp, ecg_inp])
                        y_test_preop.append(int(float(row['NRS'])))

                    # 해당 caseid가 train set에 해당하는 경우
                    elif row['caseids'] in caseid_val:
                        age_val_preop.append(int(row_demo['age']))
                        if row_demo['gender']=='F':
                            gender_val_preop.append(1)
                        else:
                            gender_val_preop.append(0)                    
                        x_val_preop.append([pleth_inp, ecg_inp])
                        y_val_preop.append(int(float(row['NRS'])))

                    elif row['caseids'] in caseid_train:
                        age_train_preop.append(int(row_demo['age']))
                        if row_demo['gender']=='F':
                            gender_train_preop.append(1)
                        else:
                            gender_train_preop.append(0)                    
                        x_train_preop.append([pleth_inp, ecg_inp])
                        y_train_preop.append(int(float(row['NRS'])))
                        
                    else:
                        non_list.append(row['caseids'])

            print('completed')

    x_train_preop = np.array(x_train_preop, np.float32)
    x_test_preop = np.array(x_test_preop, np.float32)
    y_train_preop = np.array(y_train_preop, int)
    y_test_preop = np.array(y_test_preop, int)
    x_val_preop = np.array(x_val_preop, np.float32)
    y_val_preop = np.array(y_val_preop, int)

    age_train_preop = np.array(age_train_preop, int)
    age_test_preop = np.array(age_test_preop, int)
    age_val_preop = np.array(age_val_preop, int)
    gender_train_preop = np.array(gender_train_preop, int)
    gender_test_preop = np.array(gender_test_preop, int)
    gender_val_preop = np.array(gender_val_preop, int)


    # 저장하기
    print('saving...', end='', flush=True)
    np.savez_compressed(input_path+'x_train_preop2.npz', x_train_preop)
    np.savez_compressed(input_path+'x_test_preop2.npz', x_test_preop)
    np.savez_compressed(input_path+'x_val_preop2.npz', x_val_preop)
    np.savez_compressed(input_path+'y_train_preop2.npz', y_train_preop)
    np.savez_compressed(input_path+'y_test_preop2.npz', y_test_preop)
    np.savez_compressed(input_path+'y_val_preop2.npz', y_val_preop)

    np.savez_compressed(input_path+'age_train_preop2.npz', age_train_preop)
    np.savez_compressed(input_path+'age_test_preop2.npz', age_test_preop)
    np.savez_compressed(input_path+'age_val_preop2.npz', age_val_preop)    
    np.savez_compressed(input_path+'gender_train_preop2.npz', gender_train_preop)
    np.savez_compressed(input_path+'gender_test_preop2.npz', gender_test_preop)
    np.savez_compressed(input_path+'gender_val_preop2.npz', gender_val_preop)    

    print('done', flush=True)

    
    
    
print('size of training set(preop):', len(x_train_preop))
print('size of validation set(preop):', len(x_val_preop))
print('size of test set(preop):', len(x_test_preop))

loading data 1901/3861 ...completed
loading data 1902/3861 ...completed
loading data 1903/3861 ...completed
loading data 1904/3861 ...completed
loading data 1905/3861 ...completed
loading data 1906/3861 ...completed
loading data 1907/3861 ...completed
loading data 1908/3861 ...completed
loading data 1909/3861 ...completed
loading data 1910/3861 ...completed
loading data 1911/3861 ...completed
loading data 1912/3861 ...completed
loading data 1913/3861 ...completed
loading data 1914/3861 ...completed
loading data 1915/3861 ...completed
loading data 1916/3861 ...completed
loading data 1917/3861 ...completed
loading data 1918/3861 ...completed
loading data 1919/3861 ...completed
loading data 1920/3861 ...completed
loading data 1921/3861 ...completed
loading data 1922/3861 ...completed
loading data 1923/3861 ...completed
loading data 1924/3861 ...completed
loading data 1925/3861 ...completed
loading data 1926/3861 ...completed
loading data 1927/3861 ...completed
loading data 1928/3861 ...co

loading data 2130/3861 ...completed
loading data 2131/3861 ...completed
loading data 2132/3861 ...completed
loading data 2133/3861 ...completed
loading data 2134/3861 ...completed
loading data 2135/3861 ...completed
loading data 2136/3861 ...completed
loading data 2137/3861 ...completed
loading data 2138/3861 ...completed
loading data 2139/3861 ...completed
loading data 2140/3861 ...completed
loading data 2141/3861 ...completed
loading data 2142/3861 ...completed
loading data 2143/3861 ...completed
loading data 2144/3861 ...completed
loading data 2145/3861 ...completed
loading data 2146/3861 ...completed
loading data 2147/3861 ...completed
loading data 2148/3861 ...completed
loading data 2149/3861 ...completed
loading data 2150/3861 ...completed
loading data 2151/3861 ...completed
loading data 2152/3861 ...completed
loading data 2153/3861 ...completed
loading data 2154/3861 ...completed
loading data 2155/3861 ...completed
loading data 2156/3861 ...completed
loading data 2157/3861 ...co

loading data 2586/3861 ...completed
loading data 2587/3861 ...completed
loading data 2588/3861 ...completed
loading data 2589/3861 ...completed
loading data 2590/3861 ...completed
loading data 2591/3861 ...completed
loading data 2592/3861 ...completed
loading data 2593/3861 ...completed
loading data 2594/3861 ...completed
loading data 2595/3861 ...completed
loading data 2596/3861 ...completed
loading data 2597/3861 ...completed
loading data 2598/3861 ...completed
loading data 2599/3861 ...completed
loading data 2600/3861 ...completed
loading data 2601/3861 ...completed
loading data 2602/3861 ...completed
loading data 2603/3861 ...completed
loading data 2604/3861 ...completed
loading data 2605/3861 ...completed
loading data 2606/3861 ...completed
loading data 2607/3861 ...completed
loading data 2608/3861 ...completed
loading data 2609/3861 ...completed
loading data 2610/3861 ...completed
loading data 2611/3861 ...completed
loading data 2612/3861 ...completed
loading data 2613/3861 ...co

loading data 2814/3861 ...completed
loading data 2815/3861 ...completed
loading data 2816/3861 ...completed
loading data 2817/3861 ...completed
loading data 2818/3861 ...completed
loading data 2819/3861 ...completed
loading data 2820/3861 ...completed
loading data 2821/3861 ...completed
loading data 2822/3861 ...completed
loading data 2823/3861 ...completed
loading data 2824/3861 ...completed
loading data 2825/3861 ...completed
loading data 2826/3861 ...completed
loading data 2827/3861 ...completed
loading data 2828/3861 ...completed
loading data 2829/3861 ...completed
loading data 2830/3861 ...completed
loading data 2831/3861 ...completed
loading data 2832/3861 ...completed
loading data 2833/3861 ...completed
loading data 2834/3861 ...completed
loading data 2835/3861 ...completed
loading data 2836/3861 ...completed
loading data 2837/3861 ...completed
loading data 2838/3861 ...completed
loading data 2839/3861 ...completed
loading data 2840/3861 ...completed
loading data 2841/3861 ...co

loading data 3042/3861 ...completed
loading data 3043/3861 ...completed
loading data 3044/3861 ...completed
loading data 3045/3861 ...completed
loading data 3046/3861 ...completed
loading data 3047/3861 ...completed
loading data 3048/3861 ...completed
loading data 3049/3861 ...completed
loading data 3050/3861 ...completed
loading data 3051/3861 ...completed
loading data 3052/3861 ...completed
loading data 3053/3861 ...completed
loading data 3054/3861 ...completed
loading data 3055/3861 ...completed
loading data 3056/3861 ...completed
loading data 3057/3861 ...completed
loading data 3058/3861 ...completed
loading data 3059/3861 ...completed
loading data 3060/3861 ...completed
loading data 3061/3861 ...completed
loading data 3062/3861 ...completed
loading data 3063/3861 ...completed
loading data 3064/3861 ...completed
loading data 3065/3861 ...completed
loading data 3066/3861 ...completed
loading data 3067/3861 ...completed
loading data 3068/3861 ...completed
loading data 3069/3861 ...co

loading data 3505/3861 ...completed
loading data 3506/3861 ...completed
loading data 3507/3861 ...completed
loading data 3508/3861 ...completed
loading data 3509/3861 ...completed
loading data 3510/3861 ...completed
loading data 3511/3861 ...completed
loading data 3512/3861 ...completed
loading data 3513/3861 ...completed
loading data 3514/3861 ...completed
loading data 3515/3861 ...completed
loading data 3516/3861 ...completed
loading data 3517/3861 ...completed
loading data 3518/3861 ...completed
loading data 3519/3861 ...completed
loading data 3520/3861 ...completed
loading data 3521/3861 ...completed
loading data 3522/3861 ...completed
loading data 3523/3861 ...completed
loading data 3524/3861 ...completed
loading data 3525/3861 ...completed
loading data 3526/3861 ...completed
loading data 3527/3861 ...completed
loading data 3528/3861 ...completed
loading data 3529/3861 ...completed
loading data 3530/3861 ...completed
loading data 3531/3861 ...completed
loading data 3532/3861 ...co

In [1]:
import statsmodels.api as sm


def smooth(y):
    #return savitzky_golay(y, window_size=2001, order=3)
    return lowess(y)

# 0.2가 제일 잘 없앴음
def lowess(y, f=0.2):
    x = np.arange(0, len(y))
    return sm.nonparametric.lowess(y, x, frac=f, it=0)[:, 1].T

import numpy as np
import pandas as pd
import os, pickle, sys
from scipy import signal
#sys.path.append('DL_model')


### input 설정
SRATE = 250 # 250Hz
LEN_INPUT = 20 # input 10s
LEN_PER_NRS = 300 # vital length for each NRS
OVERLAP = 5
n_aug = int((LEN_PER_NRS-LEN_INPUT)/OVERLAP) + 1 # data augmentation 개수



input_path = '../DL_model/dataset/preprocess5/'
if not os.path.exists(input_path[:-1]):
    os.mkdir(input_path[:-1])

if os.path.exists(input_path+'x_train_pacu.npz'):
    print('loading train...', flush=True, end='')
    x_train_pacu = np.load(input_path+'x_train_pacu.npz', allow_pickle=True)['arr_0']
    y_train_pacu = np.load(input_path+'y_train_pacu.npz')['arr_0']
    x_test_pacu = np.load(input_path+'x_test_pacu.npz', allow_pickle=True)['arr_0']
    y_test_pacu = np.load(input_path+'y_test_pacu.npz')['arr_0']
    print('done', flush=True)

else:
    false_row_list_preop = []
    
    vital_path = '../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/'
    ecg_path = '../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'
    df_preprocess_pacu = pickle.load(open('../DL_model/cache/preprocess5/df_preprocess_pacu_agender','rb'))
    df_demograph = pickle.load(open('../DL_model/df_caseids+age+gender', 'rb'))
    

    ### test set에 해당하는 file_path
    caseid_test = pickle.load(open('../DL_model/caseid_test_new', 'rb'))
    caseid_train = pickle.load(open('../DL_model/caseid_train_new', 'rb'))
    caseid_val = pickle.load(open('../DL_model/caseid_val_new', 'rb'))
    
    
    non_lis = []
    x_train_pacu, y_train_pacu = [], []
    x_test_pacu, y_test_pacu = [], []
    x_val_pacu, y_val_pacu = [], []
    age_train_pacu, gender_train_pacu = [], []
    age_test_pacu, gender_test_pacu = [], []
    age_val_pacu, gender_val_pacu = [], []

    
    cnt = 0
    for _, row in df_preprocess_pacu.iterrows():
        cnt += 1
        if cnt<=4000:
            continue
        if cnt>5000:
            break
            
            
        print('loading data {}/{} ...'.format(cnt, len(df_preprocess_pacu)), end='')
        row_demo = df_demograph[df_demograph['caseids']==row['caseids']].iloc[0]
            

        LEN = LEN_PER_NRS*SRATE
        # vital data - PPG (resampling 100 Hz to 250 Hz)
        df_vital = pickle.load(open(vital_path+row['file_path'], 'rb')).reset_index()
        pleth_samp = df_vital[['PPG']].fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        pleth_resamp = signal.resample(pleth_samp, LEN)
        #ppg_per_NRS = np.full(LEN, np.nan)
        #ppg_per_NRS[-len(pleth_resamp):] = pleth_resamp
        ppg_per_NRS = pleth_resamp
        

        # vital data - ECG (250Hz)
        ecg_samp = pickle.load(open(ecg_path+row['file_path'], 'rb')).reset_index()[['ECG']]
        ecg_samp = ecg_samp.fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        #ecg_per_NRS = np.full(LEN,np.nan)
        #ecg_per_NRS[-len(ecg_samp):] = ecg_samp
        ecg_per_NRS = ecg_samp
        
   
        if len(pleth_resamp) != LEN or len(ecg_samp) != LEN:
            print(row['caseids'])
            break
            

        save_path = '../../cranberry2/Preprocessing/cache/lowess_filtered/preprocess5/pacu_'+row['file_path']
        # 한 NRS에 대해 23개의 input 확인
        for i in range(23,57):
            # input이 전처리 통과한 경우
            if row[str(i+1)][0]:
                if os.path.exists(save_path+'_{}'.format(i)):
                    ppg_inp, ecg_inp = pickle.load(open(save_path+'_{}'.format(i), 'rb'))
                
                else:
                    start_idx = i*OVERLAP*SRATE # 500i
                    end_idx = (i*OVERLAP+LEN_INPUT)*SRATE # 500i + 1000

                    # PPG smoothing
                    ppg_inp = np.copy(ppg_per_NRS[start_idx:end_idx])
                    if np.sum(np.isnan(ppg_inp))!=0:
                        ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
                    ppg_inp = ppg_inp - smooth(ppg_inp)
                    
                    # ECG smoothing
                    ecg_inp = np.copy(ecg_per_NRS[start_idx:end_idx])
                    if np.sum(np.isnan(ecg_inp))!=0:
                        ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
                    ecg_inp = ecg_inp - smooth(ecg_inp)                    
                
                    # pickle에 저장
                    pickle.dump([ppg_inp, ecg_inp],  open(save_path+'_{}'.format(i), 'wb'))
                    print('{}...'.format(i+1), end='')
        print('done')
'''                
                # normalization
                pleth_inp = ppg_inp - np.nanmean(ppg_inp)
                ecg_inp2 = (ecg_inp - np.nanmean(ecg_inp)) / np.nanstd(ecg_inp)
                
                
                # 해당 caseid가 test set에 속하는 경우
                if row['caseids'] in caseid_test:
                    age_test_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_test_pacu.append(1)
                    else:
                        gender_test_pacu.append(0)
                    x_test_pacu.append([pleth_inp, ecg_inp2])
                    y_test_pacu.append(int(float(row['NRS'])))

                # 해당 caseid가 train set에 해당하는 경우
                elif row['caseids'] in caseid_val:
                    age_val_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_val_pacu.append(1)
                    else:
                        gender_val_pacu.append(0)                    
                    x_val_pacu.append([pleth_inp, ecg_inp2])
                    y_val_pacu.append(int(float(row['NRS'])))
                    
                elif row['caseids'] in caseid_train:
                    age_train_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_train_pacu.append(1)
                    else:
                        gender_train_pacu.append(0)                    
                    x_train_pacu.append([pleth_inp, ecg_inp2])
                    y_train_pacu.append(int(float(row['NRS'])))
                    
                else:
                    non_lis.append(row['caseids'])
                    
        print('completed')
        
        
    x_train_pacu = np.array(x_train_pacu, np.float32)
    x_test_pacu = np.array(x_test_pacu, np.float32)
    y_train_pacu = np.array(y_train_pacu, int)
    y_test_pacu = np.array(y_test_pacu, int)
    x_val_pacu = np.array(x_val_pacu, np.float32)
    y_val_pacu = np.array(y_val_pacu, int)
    
    age_train_pacu = np.array(age_train_pacu, int)
    age_test_pacu = np.array(age_test_pacu, int)
    age_val_pacu = np.array(age_val_pacu, int)
    gender_train_pacu = np.array(gender_train_pacu, int)
    gender_test_pacu = np.array(gender_test_pacu, int)
    gender_val_pacu = np.array(gender_val_pacu, int)
    
        
    # 저장하기
    print('saving...', end='', flush=True)
    np.savez_compressed(input_path+'x_train_pacu2.npz', x_train_pacu)
    np.savez_compressed(input_path+'x_test_pacu2.npz', x_test_pacu)
    np.savez_compressed(input_path+'x_val_pacu2.npz', x_val_pacu)
    np.savez_compressed(input_path+'y_train_pacu2.npz', y_train_pacu)
    np.savez_compressed(input_path+'y_test_pacu2.npz', y_test_pacu)
    np.savez_compressed(input_path+'y_val_pacu2.npz', y_val_pacu)
    
    np.savez_compressed(input_path+'age_train_pacu2.npz', age_train_pacu)
    np.savez_compressed(input_path+'age_test_pacu2.npz', age_test_pacu)
    np.savez_compressed(input_path+'age_val_pacu2.npz', age_val_pacu)    
    np.savez_compressed(input_path+'gender_train_pacu2.npz', gender_train_pacu)
    np.savez_compressed(input_path+'gender_test_pacu2.npz', gender_test_pacu)
    np.savez_compressed(input_path+'gender_val_pacu2.npz', gender_val_pacu)    
    
    print('done', flush=True)

    
    
    
print('size of training set(pacu):', len(x_train_pacu))
print('size of validation set(pacu):', len(x_val_pacu))
print('size of test set(pacu):', len(x_test_pacu))
'''

loading data 4001/6963 ...24...25...45...47...48...57...done
loading data 4002/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 4003/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 4004/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...done
loading data 4005/6963 ...24...25...26...34...35...36...37...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 4006/6963 ...30...31...32...41...42...43...44...45...46...47...48...49...50...51...52...53...54...done
loading data 4007/6963 ...24...25...26...31...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...done
loading data 4008/

loading data 4072/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...54...done
loading data 4073/6963 ...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...47...48...49...50...51...52...53...54...55...56...done
loading data 4074/6963 ...24...25...26...27...32...33...34...35...36...37...38...39...40...41...42...55...56...57...done
loading data 4075/6963 ...done
loading data 4076/6963 ...30...done
loading data 4077/6963 ...32...33...35...43...55...56...done
loading data 4078/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 4079/6963 ...31...32...34...35...done
loading data 4080/6963 ...31...32...33...45...50...done
loading data 4081/6963 ...done
loading data 4082/6963 ...24...25...26...27...28...29...30...31...32...33...34..

loading data 4132/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...49...50...51...52...53...54...55...56...57...done
loading data 4133/6963 ...24...25...26...27...28...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 4134/6963 ...26...27...42...43...44...45...46...50...51...52...53...54...55...56...57...done
loading data 4135/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...57...done
loading data 4136/6963 ...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 4137/6963 ...34...35...36...37...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 4138/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44.

loading data 4190/6963 ...24...25...26...27...28...29...30...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 4191/6963 ...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...45...46...done
loading data 4192/6963 ...37...38...41...44...done
loading data 4193/6963 ...24...25...26...27...28...29...30...51...56...57...done
loading data 4194/6963 ...27...28...33...35...36...37...38...39...40...41...42...43...44...45...46...47...50...51...52...done
loading data 4195/6963 ...28...29...30...31...32...33...50...51...52...53...54...55...56...57...done
loading data 4196/6963 ...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...52...54...55...56...57...done
loading data 4197/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...47...48...49...50...51...52...53...done
loading data 4198/6963 ...32...33...34...3

loading data 4262/6963 ...30...31...32...33...34...35...36...37...38...39...40...41...42...47...48...49...done
loading data 4263/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...48...49...50...51...52...done
loading data 4264/6963 ...24...25...26...27...28...30...35...36...37...38...39...40...41...42...43...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 4265/6963 ...26...27...28...29...30...31...32...33...34...35...36...38...39...42...43...44...45...46...47...48...49...done
loading data 4266/6963 ...24...25...26...27...28...29...30...31...32...33...38...39...40...41...42...43...44...45...50...51...52...53...54...55...56...57...done
loading data 4267/6963 ...24...25...26...31...32...33...34...39...47...48...49...50...51...52...53...54...55...56...57...done
loading data 4268/6963 ...34...35...46...47...48...49...50...51...52...53...54...56...done
loading data 4269/6963 ...24...31...32...33

...

loading data 4732/6963 ...55...done
loading data 4733/6963 ...24...25...29...30...31...37...38...39...40...44...45...done
loading data 4734/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...41...43...44...45...46...47...48...50...51...52...53...54...55...56...57...done
loading data 4735/6963 ...28...34...35...36...37...38...39...40...41...42...47...48...51...52...53...54...55...56...done
loading data 4736/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...39...40...41...42...43...44...45...46...47...48...49...50...57...done
loading data 4737/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...38...40...41...50...52...54...55...56...57...done
loading data 4738/6963 ...24...25...26...27...35...36...45...46...47...48...49...50...51...52...53...54...55...56...done
loading data 4739/6963 ...31...32...33...34...35...51...52...53...54...55...56...57...done
loading data 4740/6963 ...24...25...26...27...28...2

loading data 4802/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 4803/6963 ...27...28...29...30...31...32...33...34...35...37...39...40...41...42...43...44...45...46...47...48...49...51...52...53...54...55...56...57...done
loading data 4804/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...done
loading data 4805/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 4806/6963 ...done
loading data 4807/6963 ...37...38...done
loading data 4808/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51.

loading data 4871/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...46...47...48...49...50...51...52...53...54...55...56...done
loading data 4872/6963 ...24...25...29...31...32...33...34...39...40...41...42...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 4873/6963 ...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...49...50...51...57...done
loading data 4874/6963 ...27...28...29...30...31...32...33...34...35...39...43...44...45...46...47...48...52...53...54...done
loading data 4875/6963 ...24...25...26...31...32...33...34...35...36...37...38...39...40...43...44...45...46...47...48...54...55...56...57...done
loading data 4876/6963 ...24...25...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 4877/6963 ...24...25...26...27...28...29...30...31...32...33...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55.

loading data 4936/6963 ...30...38...39...40...41...43...44...done
loading data 4937/6963 ...42...done
loading data 4938/6963 ...done
loading data 4939/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...50...51...52...53...54...55...56...57...done
loading data 4940/6963 ...24...25...done
loading data 4941/6963 ...33...done
loading data 4942/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 4943/6963 ...26...27...28...29...33...34...35...39...40...41...55...56...57...done
loading data 4944/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 4945/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...43...44...45...46...47...48...49...50...

"                \n                # normalization\n                pleth_inp = ppg_inp - np.nanmean(ppg_inp)\n                ecg_inp2 = (ecg_inp - np.nanmean(ecg_inp)) / np.nanstd(ecg_inp)\n                \n                \n                # 해당 caseid가 test set에 속하는 경우\n                if row['caseids'] in caseid_test:\n                    age_test_pacu.append(int(row_demo['age']))\n                    if row_demo['gender']=='F':\n                        gender_test_pacu.append(1)\n                    else:\n                        gender_test_pacu.append(0)\n                    x_test_pacu.append([pleth_inp, ecg_inp2])\n                    y_test_pacu.append(int(float(row['NRS'])))\n\n                # 해당 caseid가 train set에 해당하는 경우\n                elif row['caseids'] in caseid_val:\n                    age_val_pacu.append(int(row_demo['age']))\n                    if row_demo['gender']=='F':\n                        gender_val_pacu.append(1)\n                    else:\n          